# Homework 2 - IEOR 224

## Introduction
This assignment contains two parts. In the first six questions, we use mean-variance optimization to construct a portfolio of sector ETFs of the S&P500. The remaining four questions are from the book. I uploaded screenshots of the questions to bCourses for your convenience.

For formatting your textual answers, you can use math mode as well as other Markdown options. [More details on formatting are found here](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html).

## Setup

In [1]:
# DO NOT CHANGE. Autograder may fail otherwise.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import fix_yahoo_finance
import datetime
import os


In [2]:
import cvxpy as cp

In [3]:
# DO NOT CHANGE. Autograder may fail otherwise.
try:
    if os.environ['GRADE'] == 'TRUE':
        print('Entering grade mode')
        grade = True
    else:
        grade = False
except KeyError:
    grade = False

## Data

We will use the 10 S&P500 sector ETFs for analysis. Furthermore, we will assume that the risk free rate is 2.6%. This was the 1-year T-bill yield on January 2nd, 2019.

In [4]:
# SP500 sector ETFs
tickers = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY']
risk_free_rate = 0.026 # 1-year T-bill on 1 Jan 2019

In [5]:
def get_adj_closing_prices(tickers, start_date, end_date):
    all_prices = {}
    for ticker in tickers:
        prices = pdr.get_data_yahoo(ticker, 
            start=start_date,
            end=end_date
        )
        all_prices[ticker] = prices["Adj Close"]
    
    return pd.DataFrame(all_prices)

In [6]:
if not grade:
    # get daily price data 2016-2018
    prices = get_adj_closing_prices(tickers, datetime.datetime(2015, 12, 31), datetime.datetime(2018, 12, 31))

In [7]:
if not grade:
    # compute daily returns
    daily_returns = prices.pct_change().dropna()

In [8]:
if not grade:
    print(daily_returns.head())

                 XLB       XLE       XLF       XLI       XLK       XLP  \
Date                                                                     
2016-01-04 -0.015661 -0.000332 -0.019303 -0.013394 -0.013075 -0.012676   
2016-01-05 -0.000468  0.003814  0.003851  0.002677 -0.002602  0.006419   
2016-01-06 -0.026217 -0.038493 -0.015345 -0.015446 -0.012334 -0.003388   
2016-01-07 -0.027163 -0.024399 -0.028139 -0.027116 -0.029539 -0.012000   
2016-01-08 -0.010131 -0.012857 -0.015590 -0.010153 -0.007919 -0.007692   

                XLRE       XLU       XLV       XLY  
Date                                                
2016-01-04 -0.023217 -0.002079 -0.018048 -0.017144  
2016-01-05  0.030302  0.007178  0.004807 -0.001302  
2016-01-06 -0.009655 -0.001839 -0.008161 -0.009776  
2016-01-07 -0.018340 -0.006679 -0.020287 -0.020534  
2016-01-08 -0.013111 -0.000464 -0.015059 -0.010751  


## Question 1
Complete function to compute the expected daily returns for each of the sector ETFs. You may assume that expected returns are given by the arithmetic average of the historical daily returns in the dataset.

In [9]:
def compute_expected_daily_returns(df):
    """Computes expected daily return for each security.
    
    Args:
        df: (pd.DataFrame): DataFrame with daily returns for a set of securities. Each column corresponds to a security.
                            Observations are indexed by date.
    
    Returns:
        np.array with expected daily return for each security.
    """
    daily_returns = np.mean(df,axis=0)
    # TODO
    return daily_returns

In [10]:
if not grade:
    exp_returns_daily = compute_expected_daily_returns(daily_returns)
    # convert average daily returns to annualized returns
    exp_returns_annual = np.power(1 + exp_returns_daily, 252) - 1

In [11]:
# print formatted expected returns
if not grade:
    print("Annualized Returns")
    for ticker, x in zip(tickers, exp_returns_annual):
        print("%-4s % 5.2f%%" % (ticker, x * 100))

Annualized Returns
XLB   8.68%
XLE   3.43%
XLF   22.74%
XLI   10.12%
XLK   16.69%
XLP   3.63%
XLRE  4.71%
XLU   11.66%
XLV   9.10%
XLY   10.97%


## Question 2
Complete the function to compute the covariance matrix of the daily returns of the sector ETFs. You may use the historical daily returns to estimate the covariance matrix.

In [12]:
def compute_daily_covariance_matrix(df):
    """Computes covariance matrix of daily returns
    
    Args:
        df: (pd.DataFrame): DataFrame with daily returns for a set of securities. Each column corresponds to a security.
                            Observations are indexed by date.
    
    Returns:
        np.array of size n x n where n is the number of securities.    
    
    """
    covs = np.cov(np.array(df.T))
    return covs

In [13]:
if not grade:
    cov_daily = compute_daily_covariance_matrix(daily_returns)
    cov_matrix_annual = cov_daily * 252 # Annualize: multiply with number of trading days in a year

In [14]:
# print formatted covariance matrix
if not grade:
    print("Annualized Covariance Matrix")
    print(' ' * 7, end="")
    for tick in tickers:
        print("% 7s" % tick, end='')
    print('\n', end="")
    print(' ' * 5, end="")
    print('-' * 72)
    for i, row_ticker in enumerate(tickers):
        print('%-4s | ' % row_ticker, end='')
        for j, col_ticker in enumerate(tickers):
            print("% 6.4f" % cov_matrix_annual[i,j], end='')
        print('\n', end='')

Annualized Covariance Matrix
           XLB    XLE    XLF    XLI    XLK    XLP   XLRE    XLU    XLV    XLY
     ------------------------------------------------------------------------
XLB  |  0.0264 0.0231 0.0207 0.0205 0.0193 0.0092 0.0099 0.0033 0.0148 0.0173
XLE  |  0.0231 0.0426 0.0210 0.0196 0.0185 0.0085 0.0092 0.0036 0.0146 0.0170
XLF  |  0.0207 0.0210 0.0621 0.0210 0.0189 0.0081 0.0093 0.0012 0.0158 0.0178
XLI  |  0.0205 0.0196 0.0210 0.0221 0.0191 0.0092 0.0092 0.0034 0.0150 0.0172
XLK  |  0.0193 0.0185 0.0189 0.0191 0.0295 0.0102 0.0109 0.0044 0.0177 0.0214
XLP  |  0.0092 0.0085 0.0081 0.0092 0.0102 0.0136 0.0103 0.0090 0.0089 0.0094
XLRE |  0.0099 0.0092 0.0093 0.0092 0.0109 0.0103 0.0220 0.0119 0.0086 0.0102
XLU  |  0.0033 0.0036 0.0012 0.0034 0.0044 0.0090 0.0119 0.0191 0.0039 0.0033
XLV  |  0.0148 0.0146 0.0158 0.0150 0.0177 0.0089 0.0086 0.0039 0.0205 0.0147
XLY  |  0.0173 0.0170 0.0178 0.0172 0.0214 0.0094 0.0102 0.0033 0.0147 0.0216


## Question 3
Complete the following three functions to compute the expected return, the variance, and the standard deviation of a portfolio given the assets' expected return and standard deviation and the weights of the assets. 

In [15]:
def compute_portfolio_expected_return(return_vector, weights):
    """Computes expected return of a portfolio given the asset returns and the asset weights in the portfolio."""
    # TODO
    return return_vector.T.dot(weights)

def compute_portfolio_variance(covariance_matrix, weights):
    """Computes the variance of a portfolio given the asset covariance matrix and the asset weights in the portfolio."""
    # TODO
    return weights.T.dot(covariance_matrix).dot(weights)

def compute_portfolio_std(covariance_matrix, weights):
    """Computes the standard deviation of a portfolio given the asset covariance matrix and the asset weights in the portfolio."""
    # TODO
    return np.sqrt(compute_portfolio_variance(covariance_matrix, weights))

## Question 4
Complete the `min_risk_portfolio` function to construct a portfolio with an expected return equal to the `target_return` and the smallest possible variance. The portfolio should be fully invested and no shorting is allowed. We use this function in the `efficient_frontier` function to compute the efficient frontier.

In [23]:
 def min_risk_portfolio(expected_returns, covariance_matrix, target_return):
    """Computes the weights of a minimum variance portfolio for a given expected return.
    The portfolio is fully vested (weights sum to 1) and no shorting is allowed.
    
    Args:
        expected_returns (np.array): Expected returns for n assets.
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        target_return (float): Expected return target for the portfolio
        
    Returns:
        np.array of length n with the weights of each asset in the portfolio
        OR
        None if no feasible portfolio exists.
    """
    n = expected_returns.shape[0]
    
    w = cp.Variable(n)                         # Portfolio allocation vector
    ret = expected_returns.T * w
    risk = cp.quad_form(w, covariance_matrix)
    target_ret = cp.Parameter()
    target_ret.value = target_return
    prob = cp.Problem(cp.Minimize(risk),          # Restricting to long-only portfolio
                   [ret == target_ret, # match target_return
                   cp.sum(w) == 1, # sum of weights in portfolios sum to 1.
                   w >= 0])
    prob.solve()
    
    if prob.status == 'optimal':
        return w.value
    else:
        return None

In [26]:
def efficient_frontier(expected_returns, covariance_matrix):
    """Construct efficient frontier portfolios by a line sweep of target returns.
    
    Args:
        expected_returns (np.array): Expected returns for n assets.
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        
    Returns:
        List of np.arrays. Each numpy array in the list represents a portfolio on the efficient frontier.
    
    """
    min_return = np.min(expected_returns)
    max_return = np.max(expected_returns)
    
    target_returns = np.linspace(min_return,max_return, num=200)
    
    portfolio_weights = []
    
    for tr in target_returns:
        result = min_risk_portfolio(expected_returns, covariance_matrix, tr)
        # only add results if optimization was successful
        if result is not None:
            weights = result
            portfolio_weights.append(weights)
           
    return portfolio_weights

In [27]:
if not grade:
    portfolio_weights = efficient_frontier(exp_returns_annual, cov_matrix_annual)

Exception: Cannot evaluate the truth value of a constraint or chain constraints, e.g., 1 >= x >= 0.

## Question 5
Complete the function to determine the minimum variance portfolio among a list of portfolios.

In [20]:
def min_variance_portfolio(covariance_matrix, portfolio_weights):
    """Select minimum variance portfolio from a list of portfolios.
    
    Args:
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        portfolio_weights (list of np.array): Each numpy array in the list represents a portfolio.
        
    Returns:
        np.array with weights of portfolio that has the smallest variance.
    """
    portfolio_weights1 = []
    portfolio_means = []
    portfolio_var = []
    for weights in portfolio_weights:
        port_var=compute_portfolio_variance(covariance_matrix, weights)
        portfolio_weights1.append(weights)
        portfolio_var.append(port_var)
    
    index = np.argmin(portfolio_var)
    weights = portfolio_weights1[index]
    return weights

In [21]:
if not grade:
    min_variance_weights = min_variance_portfolio(cov_matrix_annual, portfolio_weights)
    min_variance_expected_return = compute_portfolio_expected_return(exp_returns_annual, min_variance_weights)
    min_variance_std = compute_portfolio_std(cov_matrix_annual, min_variance_weights)

NameError: name 'portfolio_weights' is not defined

In [47]:
if not grade:
    for tick, x in zip(tickers, min_variance_weights):
        print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

NameError: name 'min_variance_weights' is not defined

## Question 6

In [64]:
def max_sharpe_portfolio(expected_returns, covariance_matrix, risk_free_rate, portfolio_weights):
    """Select max sharpe portfolio from a list of portfolios.
    
    Args:
        expected_returns (np.array): Expected returns for n assets.
        covariance_matrix (np.array): n x n Covariance matrix of asset returns.
        risk_free_rate (float): Risk free rate.
        portfolio_weights (list of np.array): Each numpy array in the list represents a portfolio.
        
    Returns:
        np.array with weights of portfolio that maximizes sharpe ratio.  
    """
    portfolio_weights2 = []
    portfolio_ratio2 = []
    for weights in portfolio_weights:
        port_mean=compute_portfolio_expected_return(expected_returns, weights)
        port_std=compute_portfolio_std(covariance_matrix, weights)
        sharpe_ratios = (port_mean-risk_free_rate)/ port_std
        portfolio_weights2.append(weights)
        portfolio_ratio2.append(sharpe_ratios)
    
    index = np.argmax(portfolio_ratio2)
    weights = portfolio_weights2[index]
    ratio = portfolio_ratio2[index]
    # TODO
    return ratio,weights

In [65]:
if not grade:
    max_sharpe_ratio, max_sharpe_weights = max_sharpe_portfolio(exp_returns_annual, cov_matrix_annual, risk_free_rate, portfolio_weights)
    max_sharpe_expected_return = compute_portfolio_expected_return(exp_returns_annual, max_sharpe_weights)
    max_sharpe_std = compute_portfolio_std(cov_matrix_annual, max_sharpe_weights)

NameError: name 'portfolio_weights' is not defined

In [ ]:
if not grade:
    for tick, x in zip(tickers, max_sharpe_weights):
         print('Weight %-4s %5.2f%%' % (tick, (x + 1e-10) * 100 ))

## Plot of portfolios

In [ ]:
asset_std = np.sqrt(np.diag(cov_matrix_annual))

In [48]:
fig = plt.figure(figsize=(7.5, 4))

# Compute missing returns and stds.
asset_stds = np.sqrt(np.diag(cov_matrix_annual))
portfolio_stds = [compute_portfolio_std(cov_matrix_annual, weights) for weights in portfolio_weights]
portfolio_returns = [compute_portfolio_expected_return(exp_returns_annual, weights) for weights in portfolio_weights]

# Plot Capital Market Line
x_range = np.linspace(0, 0.3)
plt.plot(x_range, risk_free_rate + max_sharpe_ratio * x_range , c='cyan', label="CML")

# Plot portfolios
plt.scatter(portfolio_stds, portfolio_returns, label='Efficient frontier', s=4)
plt.scatter(asset_stds, exp_returns_annual, label='Assets')
plt.scatter([0],[risk_free_rate], label='Risk free asset')
plt.scatter([min_variance_std],[min_variance_expected_return], label='Min Variance')
plt.scatter([max_sharpe_std],[max_sharpe_expected_return], label='Max Sharpe')

plt.ylim([0, 0.3])
plt.xlim([0, 0.3])

for i, txt in enumerate(tickers):
    plt.annotate(txt, (asset_std[i], exp_returns_annual[i]))

plt.xlabel('Annualized standard deviation')
plt.ylabel('Annualized expected return')
plt.legend()

NameError: name 'portfolio_weights' is not defined

<Figure size 540x288 with 0 Axes>

## Question 7
Answer question 7.12 from the Investments book.

**Answer here**
Since the correlation of A and B = -1, riskless hedge is possible. In addition, because the standard deviation of B is twice as that of A, the amount of stock A should be as twice as amount of stock B to best lower the risk. Therefore, risk-free rate = 10%×(2/3)+15%×(1/3) = 11.67%

## Question 8
Answer question 8.6 from the Investments book.

### Part a)

**Answer here**

stock A: (0.8^2)×(0.22^2) + 0.3^2 =  0.120976 and standard deviation = 34.78%

stock B: (1.2^2)×(0.22^2) + 0.4^2  = 0.229696 and standard deviation = 47.93%

### Part b)

**Answer here**

Both the expected return and beta is the weighted average of the expected return and beta of individual serurities. 

expected return = 13%×0.3 + 18%×0.45 + 8%×0.25 = 14%

beta = 0.8×0.3 +1.2×0.45 + 0 = 0.78

nonsystematic standard deviation = square root of (0.3^2×0.3^2 + 0.45^2×0.4^2 + 0.25^2×0 ) = 20.12%

portfolio standard deviation = square root of (0.78^2×0.22^2 + 0.2012^2) = 26.447%

## Question 9
Answer question 9.23 from the Investments book.

### Part a)

**Answer here**

According to CAPM, E(p) = 5% + 0.8×(15%-5%) = 13% and 14% - 13% = 1% > 0

As a result, we should invest in this fund and the fund's alpha is 1%

### Part b)

**Answer here**

The passive portfolio with the same beta as the fund should be invested 80% in the market-index portfolio and 20% in the money market account.  For this portfolio: 

E(rp) = (0.8 × 15%) + (0.2 × 5%) = 13%

14% − 13% = 1% = alpha 

## Question 10
Answer question 24.12 from the Investments book.

### Part a)

**Answer here**

Total value added of all the manager's decisions this period:

(0.3×0.2 + 0.1×0.15 + 0.4×0.1 + 0.2×0.05)- (0.15×0.12 + 0.3×0.15 + 0.45×0.14 + 0.1×0.12) = -0.013


### Part b)

**Answer here**

Value added by her country allocation decisions:

(0.3-0.15)×0.12 + (0.1-0.3)×0.15 + (0.4-0.45)×0.14 + (0.2-0.1)×0.12 = - 0.007

### Part c)

**Answer here**

value added from her stock selection ability within countries:

0.3×(0.2-0.12) + 0.1×(0.15-0.15) + 0.4×(0.1-0.14) + 0.2×(0.05- 0.12) = -0.006

### Part d)

**Answer here**

We can confirm it by proving that

(value added from her stock selection ability within countries) + (Value added by her country allocation decisions) = (Total value added of all the manager's decisions this period)

-0.007 - 0.006 = -0.013 

## Submission instructions
There are **two parts** to submit on Gradescope, a PDF of your notebook and its .py file.
1. Remove or comment out any cells or code that you added other than the original functions and code. Autograding may fail without this step.
2. Download your Jupyter Notebook as a PDF. With your Notebook pulled up in a browser, open the print menu for your browser (File > Print). Change the printer to "Save to PDF", and print (this saves your Notebook as a PDF file). Check that all of your answers are still there.
3. Next, download your Juypter Notebook as a .py file. In the menu bar, choose File > Download As > Python (.py). Note: Chrome may warn that downloading the file is dangerous. You can safely ignore the warning in this case.
4. Rename the downloaded file to `HW2.py`. **IMPORTANT!**
5. Go to the IEOR224 class in Gradescope and click on Homework 1 PDF.
6. Upload your **PDF** and tag the pages corresponding to each question and your answer. 
7. Go back to the IEOR 224 class on Gradescope  and click on Homework 1 Python. Upload the `HW2.py` file.

**Note**: You may submit as many times as you like before the submission deadline, and we will use your last submission for grading.